## VIDEO RECONSTRUCTION

See [here]() for the full tutorial.

In [ ]:
# authenticate google and save model_full and model_balanced to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
MODEL_SAVE_DIR = "/content/drive/MyDrive/tensorflow_facial_recognition_output_FINAL"

In [ ]:
! pip install tensorflow_addons face_recognition keras_vggface keras_applications

     |████████████████████████████████| 706kB 8.0MB/s 
     |████████████████████████████████| 51kB 8.9MB/s 
     |████████████████████████████████| 100.2MB 104kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566173 sha256=ee0989c58f432985977b2bd21c1f742c28679a3e01ac7142db79559d2ffaf850
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


In [ ]:
!gdown -q https://drive.google.com/uc?id=1bhR5WTXID9kTxlMmgbKwf3t8-PQUIbTo
!unzip -q /content/destinys_child_data.zip

In [ ]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)
import tensorflow_addons as tfa
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import cv2
import face_recognition
import os
import numpy as np
import tqdm
import tqdm.notebook

In [ ]:
from keras_vggface import VGGFace
from keras import Model
from keras.layers import Dense, Flatten, Softmax

class_names = ['beyonce', 'kelly', 'michelle', 'unknown']

class RecognitionModel(Model):
  def __init__(self, input_len=224, classes=len(class_names)):
    super(RecognitionModel, self).__init__()
    self.vggface = VGGFace(include_top=False, model='vgg16', weights='vggface', 
              input_tensor=None, input_shape=(input_len, input_len, 3), pooling=None,
              classes=classes)
    self.flatten = Flatten()
    self.d1 = Dense(8192, activation='relu')
    self.d2 = Dense(1024, activation='relu')
    self.dc = Dense(classes)

  def call(self, x):
    x = self.vggface(x)
    x = self.flatten(x)
    x = self.dc(self.d2(self.d1(x)))
    return x

In [ ]:
model_full = RecognitionModel()
model_full.build(input_shape=(None, 224, 224, 3))

model_balanced = RecognitionModel()
model_balanced.build(input_shape=(None, 224, 224, 3))

58916864/58909280 [==============================] - 2s 0us/step


In [ ]:
@tf.function
def eval_step_full(test_images):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  logits = model_full(test_images, training=False)
  sfmax = tf.nn.softmax(logits, axis=-1)
  predictions = tf.cast(tf.equal(sfmax, np.max(sfmax, axis=1, keepdims=True)), tf.int64)
  return predictions

@tf.function
def eval_step_balanced(test_images):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  logits = model_balanced(test_images, training=False)
  sfmax = tf.nn.softmax(logits, axis=-1)
  predictions = tf.cast(tf.equal(sfmax, np.max(sfmax, axis=1, keepdims=True)), tf.int64)
  return predictions

In [ ]:
raw_base = '/content/destinys_child_data/raw'
all_videos = [os.path.join(raw_base, f) for f in os.listdir(raw_base) if f.split('.')[-1] == 'mp4']
print(all_videos)

colors = {
    'beyonce': (181, 23, 158),
    'kelly': (114, 9, 183),
    'michelle': (76, 201, 240),
    'unknown': (255, 0, 0),
}

model_full.load_weights(os.path.join(MODEL_SAVE_DIR, 'output_models/model_full', 'model_full.ckpt'))
model_balanced.load_weights(os.path.join(MODEL_SAVE_DIR, 'output_models/model_balanced', 'model_balanced.ckpt'))

# add text centered on image

def get_canon_name(video):
  return '_'.join(video.split('.')[0].split('/')[-1].split('_'))
keep = ['Soldier', 'Emotion', 'Bootylicious', 'Cater_2_U', 'Survivor']

for video in sorted(all_videos, key=lambda x: 100 if get_canon_name(x) not in keep else keep.index(get_canon_name(x))):
    SUBDIR_FULL = 'FULL'
    SUBDIR_BAL = 'BALANCED'
    canon_name = get_canon_name(video)
    if canon_name not in keep:
      continue
    new_path_f = os.path.join(MODEL_SAVE_DIR, 'VIDEO_FRAMES', SUBDIR_FULL, canon_name)
    new_path_b = os.path.join(MODEL_SAVE_DIR, 'VIDEO_FRAMES', SUBDIR_BAL, canon_name)
    os.makedirs(new_path_f, exist_ok=True)
    os.makedirs(new_path_b, exist_ok=True)
    print(canon_name)
    cap = cv2.VideoCapture(video)
    width  = cap.get(cv2.CAP_PROP_FRAME_WIDTH)   # float `width`
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    success, image = cap.read()
    total = 0
    pbar = tqdm.notebook.tqdm(total=frames)
    font = cv2.FONT_HERSHEY_SIMPLEX
    header_full = "FULL DATASET"
    header_bal = "BALANCED DATASET"
    # get boundary of this text
    textsize_full = cv2.getTextSize(header_full, font, 0.5, 2)[0]
    textsize_bal = cv2.getTextSize(header_bal, font, 0.5, 2)[0]
    # get coords based on boundary
    textX_full = int((image.shape[1] - textsize_full[0]) / 2)
    textY_full = int((image.shape[0] + textsize_full[1]) / 2 - (4 * height / 10))
    textX_bal = int((image.shape[1] - textsize_bal[0]) / 2)
    textY_bal = int((image.shape[0] + textsize_bal[1]) / 2 - (4 * height / 10))
    while success:
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      image_full = image.copy()
      image_bal = image.copy()
      cv2.putText(image_full, header_full, (textX_full, textY_full), font, 0.5, (255, 255, 255), 2)
      cv2.putText(image_bal, header_bal, (textX_full, textY_full), font, 0.5, (255, 255, 255), 2)
      faces = face_recognition.face_locations(image)
      if len(faces) > 0:
        im_orig = image.copy()
        for y1, x1, y2, x2 in faces:
          face_im = im_orig[y1:y2, x2:x1].copy()
          face_im = keras.preprocessing.image.smart_resize(face_im, (224, 224))
          face_im.resize((1, 224, 224, 3))
          one_hot = eval_step_full(face_im)
          prediction = np.argmax(one_hot)
          pred_name = class_names[prediction]
          cv2.rectangle(image_full, (x1, y1), (x2, y2), colors[pred_name], 2)
          cv2.putText(image_full, pred_name,(x2, y1-5), font,  
                    0.55, colors[pred_name], 1, cv2.LINE_AA)
          one_hot = eval_step_balanced(face_im)
          prediction = np.argmax(one_hot)
          pred_name = class_names[prediction]
          cv2.rectangle(image_bal, (x1, y1), (x2, y2), colors[pred_name], 2)
          cv2.putText(image_bal, pred_name,(x2, y1-5), font,  
                    0.55, colors[pred_name], 1, cv2.LINE_AA)
          
      plt.imsave(os.path.join(new_path_f ,'frame_%d.jpg' % total), image_full)
      plt.imsave(os.path.join(new_path_b ,'frame_%d.jpg' % total), image_bal)
      success, image = cap.read()
      total += 1
      pbar.update(1)
    cap.release()
    print('Completed %s' % canon_name)  

['/content/destinys_child_data/raw/Soldier.mp4', '/content/destinys_child_data/raw/Cater_2_U.mp4', '/content/destinys_child_data/raw/Bootylicious.mp4', '/content/destinys_child_data/raw/With_Me_Part_1.mp4', '/content/destinys_child_data/raw/Me_Myself_and_I.mp4', '/content/destinys_child_data/raw/Stand_Up_For_Love.mp4', '/content/destinys_child_data/raw/Survivor.mp4', '/content/destinys_child_data/raw/No_No_No.mp4', '/content/destinys_child_data/raw/Emotion.mp4', '/content/destinys_child_data/raw/Lose_My_Breath.mp4', '/content/destinys_child_data/raw/Bug_A_Boo.mp4', '/content/destinys_child_data/raw/Bills_Bills_Bills.mp4', '/content/destinys_child_data/raw/8_Days_of_Christmas.mp4', '/content/destinys_child_data/raw/Nasty_Girl.mp4', '/content/destinys_child_data/raw/Independent_Women_Part_1.mp4', '/content/destinys_child_data/raw/Get_On_The_Bus.mp4', '/content/destinys_child_data/raw/Say_My_Name.mp4', '/content/destinys_child_data/raw/Jumpin_Jumpin.mp4']
Soldier


Completed Soldier
Emotion


Completed Emotion
Bootylicious


Completed Bootylicious
Cater_2_U


Completed Cater_2_U
Survivor


Completed Survivor
